In [ ]:
from config import SEGEMENTATION_PARAMS
SEGEMENTATION_PARAMS[ModelEnum.OMNIPOSE]

In [1]:
from core import process, ModelEnum
process("/Users/sherryyang/Documents/wiggins-lab/data/1024/xy5/phase/", hypermodel=ModelEnum.OMNIPOSE, submodel = 'bact_phase_omni')

2024-01-06 21:33:31,286 [INFO] ** TORCH GPU version installed and working. **
2024-01-06 21:33:31,287 [INFO] >>bact_phase_omni<< model set to be used
2024-01-06 21:33:31,288 [INFO] ** TORCH GPU version installed and working. **
2024-01-06 21:33:31,288 [INFO] >>>> using GPU


/Users/sherryyang/anaconda3/envs/omnipose-track/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-01-06 21:34:13,139 [INFO] 100%|##########| 141/141 [00:41<00:00,  3.40it/s]
